# Enhanced Table
Let's keep a record of all of the features that we add in the other noteboks that would be useful to have in a consolidated table.

In [2]:
import os
from pathlib import Path
from sqlalchemy import create_engine

In [3]:
# Define your database file path (SQLite example)
rel_db_path = Path.cwd().parent.parent / 'raw_data' / 'nfl' / 'src_code' / 'nfl.db'
db_file_path = rel_db_path.resolve()
engine = create_engine(f'sqlite:///{db_file_path}')

In [4]:
%load_ext sql

In [5]:
%sql sqlite:///{db_file_path}

cleaning and prep
- cleaned name: REPLACE(REPLACE(player,'*',''),'+','') AS cleaned_player
- pro bowl status: IIF(INSTR(player, '*') > 0, 'true', 'false') AS pro_bowl
- all pro status: IIF(INSTR(player, '+') > 0, 'true', 'false') AS all_pro

per game scoring
- fantasy points per game: ROUND(fantasy_points / g) AS fppg,
- fantasy points per game STARTED: ROUND(fantasy_points / gs) AS fppgs,
- percent of games played: ROUND(g / (IIF(year >= 2021, 17, 16)*1.00),2) AS pgp
- ppr points per game: ROUND(fantasy_points_ppr / g) AS fppg_ppr,
- half ppr scoring: fantasy_points + (rec * 0.5) AS fantasy_points_hppr,

attempts & volume per game
- rush / game
- pass att / game
- 

TODOTODOTOTDTO
TODOTODOTOTDTO
volume per game stats
- completions / game
- pass yards / game
- pass td / game
- rush yards / game
- rush td / game
- rec / game
- rec yards / game
- rec td / game
- all td  / game

career stats
- first year in the data: MIN(year) OVER(PARTITION BY player_id) AS fyd,
- last year in the data: MAX(year) OVER(PARTITION BY player_id) AS lyd
- the year in our data that the player is in: year - MIN(year) OVER(PARTITION BY player_id) AS yid
- per game scoring for the NEXT year: LEAD(ROUND(fantasy_points / g)) OVER (PARTITION BY player_id ORDER BY year ASC) AS next_fppg,
- ppr version LEAD(ROUND(fantasy_points_ppr / g)) OVER (PARTITION BY player_id ORDER BY year ASC) AS next_fppg_ppr

Other/todo
- limited_participant (first year): the first year that someone scored pgp below 75%?
- should we also add "breakout" year? or "fantasy points growth"
- should we add any lag / lead to the table? or handle all of that in a programatic way?
- best fantasy finish at that position (season total) to get someone's "ceiling" 

In [37]:
%%sql
DROP TABLE IF EXISTS nfl_enhanced;
CREATE TABLE nfl_enhanced AS 
SELECT
    *,
    -- Clean up the player name
    REPLACE(REPLACE(player,'*',''),'+','') AS cleaned_player,
    IIF(INSTR(player, '*') > 0, 'true', 'false') AS pro_bowl,
    IIF(INSTR(player, '+') > 0, 'true', 'false') AS all_pro,
    
    -- Per game fantasy points & half PPR
    ROUND(fantasy_points / (g*1.0),2) AS fppg,
    ROUND(fantasy_points / (gs*1.0),2) AS fppgs,
    ROUND(fantasy_points_ppr / (g*1.0),2) AS fppg_ppr,
    fantasy_points + (rec * 0.5) AS fantasy_points_hppr,
    ROUND(g / (IIF(year >= 2021, 17, 16)*1.00),2) AS pgp,

    -- Per game attempts and volume
    ROUND(pass_att / (g*1.0),2) AS pass_att_pg,
    ROUND(rush_att / (g*1.0),2) AS rush_att_pg,
    ROUND(targets / (g*1.0),2) AS targets_pg,

    -- Per game output and rates
        -- QB
        ROUND(pass_cmp / (g*1.0),2) AS pass_cmp_pg,
        ROUND(pass_yds / (g*1.0),2) AS pass_yds_pg,
        ROUND(pass_td / (g*1.0),3) AS pass_td_pg,
        ROUND(pass_td / (pass_att*1.0),4) AS pass_td_rate,
        ROUND(pass_att / (pass_td*1.0),2) AS pass_att_ptd,
    
        -- RB    
        ROUND(rush_td / (g*1.0),3) AS rush_td_pg,
        ROUND(rush_yds / (g*1.0),2) AS rush_yds_pg,
        ROUND(rush_yds / (rush_td*1.0),2) AS rush_yds_per_td,
        ROUND(rush_att / (rush_td*1.0),2) AS rush_att_per_td,
    
        -- WR
        ROUND(rec / (g*1.0),3) AS rec_pg,
        ROUND(rec / (targets*1.0),3) AS catch_rate,
        ROUND(rec_td / (g*1.0),4) AS rec_td_pg,
        ROUND(rec_yds / (g*1.0),2) AS rec_yds_pg,
        ROUND(targets / (rec_td*1.0),2) AS tar_per_td,
        ROUND(rec / (rec_td*1.0),2) AS rec_per_td,
        ROUND(rec_yds / (rec_td*1.0),2) AS rec_yds_per_td,

    -- Career related stats
    MIN(year) OVER(PARTITION BY player_id) AS fyd,
    MAX(year) OVER(PARTITION BY player_id) AS lyd,
    year - MIN(year) OVER(PARTITION BY player_id) AS yid,
    LEAD(ROUND(fantasy_points / (g*1.0),2)) OVER (PARTITION BY player_id ORDER BY year ASC) AS next_fppg,
    ROUND(
        (LEAD(ROUND(fantasy_points / (g*1.0),2)) OVER (PARTITION BY player_id ORDER BY year ASC)
        / ROUND(fantasy_points / (g*1.0),2)) - 1
        ,4) AS next_fppg_growth,
    LEAD(ROUND(fantasy_points_ppr / g)) OVER (PARTITION BY player_id ORDER BY year ASC) AS next_fppg_ppr
FROM nfl_results

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.
Done.


[]

In [40]:
%%sql
SELECT
    * 
FROM nfl_enhanced
where 1=1 AND cleaned_player LIKE 'Tom Br%'
LIMIT 20

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


ranker,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_yds_per_att,rush_td,targets,rec,rec_yds,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,player_id,Year,cleaned_player,pro_bowl,all_pro,fppg,fppgs,fppg_ppr,fantasy_points_hppr,pgp,pass_att_pg,rush_att_pg,targets_pg,pass_cmp_pg,pass_yds_pg,pass_td_pg,pass_td_rate,pass_att_ptd,rush_td_pg,rush_yds_pg,rush_yds_per_td,rush_att_per_td,rec_pg,catch_rate,rec_td_pg,rec_yds_pg,tar_per_td,rec_per_td,rec_yds_per_td,fyd,lyd,yid,next_fppg,next_fppg_growth,next_fppg_ppr
75,Tom Brady*,NWE,QB,39,12,12,291,432,3554,28,2,28,64,2.29,0,0,0,0,None,0,5.0,0,0,None,1.0,259.0,258.6,263.6,260.6,None,15,75.0,BradTo00,2016,Tom Brady,true,false,21.58,21.58,21.55,259.0,0.75,36.0,2.33,0.0,24.25,296.17,2.333,0.0648,15.43,0.0,5.33,None,None,0.0,None,0.0,0.0,None,None,None,2016,2022,0,18.5,-0.1427,18.0
26,Tom Brady*+,NWE,QB,40,16,16,385,581,4577,32,8,25,28,1.12,0,0,0,0,None,0,7.0,3,0,None,2.0,296.0,295.9,309.9,303.9,41.0,3,26.0,BradTo00,2017,Tom Brady,true,true,18.5,18.5,18.49,296.0,1.0,36.31,1.56,0.0,24.06,286.06,2.0,0.0551,18.16,0.0,1.75,None,None,0.0,None,0.0,0.0,None,None,None,2016,2022,1,17.5,-0.0541,18.0
76,Tom Brady*,NWE,QB,41,16,16,375,570,4355,29,11,23,35,1.52,2,1,1,6,6.0,0,4.0,2,2,None,None,280.0,281.3,297.3,291.8,None,14,76.0,BradTo00,2018,Tom Brady,true,false,17.5,17.5,17.58,280.5,1.0,35.63,1.44,0.06,23.44,272.19,1.813,0.0509,19.66,0.125,2.19,17.5,11.5,0.063,1.0,0.0,0.38,None,None,None,2016,2022,2,16.5,-0.0571,16.0
76,Tom Brady,NWE,QB,42,16,16,373,613,4057,24,8,26,34,1.31,3,0,0,0,None,0,4.0,1,3,None,1.0,264.0,263.7,275.7,271.7,None,12,76.0,BradTo00,2019,Tom Brady,false,false,16.5,16.5,16.48,264.0,1.0,38.31,1.63,0.0,23.31,253.56,1.5,0.0392,25.54,0.188,2.13,11.33,8.67,0.0,None,0.0,0.0,None,None,None,2016,2022,3,21.13,0.2806,21.0
26,Tom Brady,TAM,QB,43,16,16,401,610,4633,40,12,30,6,0.2,3,0,0,0,None,0,4.0,1,3,None,None,338.0,337.9,353.9,349.9,55.0,8,26.0,BradTo00,2020,Tom Brady,false,false,21.13,21.13,21.12,338.0,1.0,38.13,1.88,0.0,25.06,289.56,2.5,0.0656,15.25,0.188,0.38,2.0,10.0,0.0,None,0.0,0.0,None,None,None,2016,2022,4,22.06,0.044,22.0
7,Tom Brady*,TAM,QB,44,17,17,485,719,5316,43,12,28,81,2.89,2,0,0,0,None,0,4.0,3,2,None,None,375.0,374.7,392.7,386.7,100.0,3,7.0,BradTo00,2021,Tom Brady,true,false,22.06,22.06,22.04,375.0,1.0,42.29,1.65,0.0,28.53,312.71,2.529,0.0598,16.72,0.118,4.76,40.5,14.0,0.0,None,0.0,0.0,None,None,None,2016,2022,5,16.0,-0.2747,16.0
73,Tom Brady,TAM,QB,45,17,17,490,733,4694,25,9,29,-1,-0.03,1,1,0,0,None,0,5.0,4,1,None,2.0,272.0,271.7,287.7,280.7,None,12,73.0,BradTo00,2022,Tom Brady,false,false,16.0,16.0,15.98,272.0,1.0,43.12,1.71,0.06,28.82,276.12,1.471,0.0341,29.32,0.059,-0.06,-1.0,29.0,0.0,0.0,0.0,0.0,None,None,None,2016,2022,6,None,None,None


In [39]:
%%sql
SELECT
    cleaned_player,
    team,
    year,
    fppg,
    next_fppg,
    next_fppg_growth
FROM nfl_enhanced
where 1=1 AND cleaned_player LIKE 'Tom Br%'
LIMIT 20

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


cleaned_player,team,Year,fppg,next_fppg,next_fppg_growth
Tom Brady,NWE,2016,21.58,18.5,-0.1427
Tom Brady,NWE,2017,18.5,17.5,-0.0541
Tom Brady,NWE,2018,17.5,16.5,-0.0571
Tom Brady,NWE,2019,16.5,21.13,0.2806
Tom Brady,TAM,2020,21.13,22.06,0.044
Tom Brady,TAM,2021,22.06,16.0,-0.2747
Tom Brady,TAM,2022,16.0,None,None
